# HSKA AI-Lab: Reinforcement Learning - Übung 01

<br>

## Mount Google Drive

(Bei Ausführung innerhalb von Google Colab)

In [2]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# %cd /content/drive/My\ Drive/ai-lab/rl

<br>

## MENACE

In diesem Notebook geht es darum einen ersten praktischen Einblick in die Implemtentierung eines simplen RL-Agenten zu erhalten.
Daher soll im folgenden eine einfache Implementierung des bereits in den Folien vorgestellten [MENACE](https://en.wikipedia.org/wiki/Matchbox_Educable_Noughts_and_Crosses_Engine), ein RL Agent zur Lösung von Tic-Tac-Toe, umgesetzt werden. Dies erfolgt mit Hilfe des RL Frameworks [OpenAI Gym](https://gym.openai.com/).

In [3]:
%pip install --upgrade pip
# %pip install gym[atari]==0.21.0
%pip install gym

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Tic-Tac-Toe Umgebung

Als ersten Schritt wollen wir die Umgebung (Environment) für MENACE modellieren, sprich das Tic-Tac-Toe Spielfeld.
Dies besteht bekanntermaßen aus einer $3 x 3$ Matrix, welche wie folgt durchnummeriert wird:
<br>

```
| 0 | 1 | 2 |
-------------
| 3 | 4 | 5 |
-------------
| 6 | 7 | 8 |
-------------
```
Der Zustand des Spiels kann also jederzeit durch ein neun-elementiges Feld mit String-Werten beschrieben werden, wobei jedes Element entweder mit `' '` für ein leeres Feld oder mit `'X'` bzw. `'O'` für gespielte Felder des jeweiligen Spielers belegt ist. Beipielsweise repräsentiert `[' ', 'X', 'O', ' ', 'X', ' ', ' ', ' ' , ' ']` die folgende Spielsituation:

```
|   | X | O |
-------------
|   | X |   |
-------------
|   |   |   |
-------------
```

Ein Agent bestimmt in seinem Zug immer das Feld, auf das seine Markierung als nächstes gesetzt wird, indem er den Index des entsprechenden Felds als Aktion wählt.
Dementsprechen ergibt sich folgender Zustands- und Aktionsraum (`action_space` und `observation_space`, s.u.):
<br>
$A = \{1,...,9\} $ <br>
$S = \{1,...,9\} $

Diese Umgebung wollen wir nun im ersten Schritt als OpenAI Gym Environment implementieren.

### Aufgabe 1: Modellierung von Tic-Tac-Toe als Environment

Dazu soll die Umgebung als eigene Klasse implementiert werden, die von `gym.Env` erbt.
Die abstrakte Klasse `Env` erfordert die Implementierung der Methoden `step`, `reset` und `render`. Hierbei beinhaltet `step` die Logik eines Spielzugs, mittels `reset` kann die Umgebung nach einem Spiel wieder zurückgesetzt werden und `render` erzeugt eine menschenlesbare Repräsentation der Umgebung.

In [4]:
from enum import Enum
from typing import List, Tuple

import gym
from gym import spaces

class Rewards(Enum):
    """Models the rewards of Tic-Tac-Toe"""
    WINNER = 3
    DRAW = 1
    NO_REWARD = 0

# size of the noughts-and-crosses board
BOARD_SIZE = 9


class NoughtsAndCrossesEnvironment(gym.Env):
    """
    Models the environment of noughts-and-crosses
    """

    def __init__(self):
        self.action_space = spaces.Discrete(BOARD_SIZE)
        self.observation_space = spaces.Discrete(BOARD_SIZE)
        # self.seed(seed=42)
        self.board = None  # representation of the board (see above)
        self.done = False
        self.current_actor_symbol = "-" # the symbol of the currently active actor
        self.reset()

    def __str__(self):
        return f"| 0 | 1 | 2 |  *  | {self.board[0]} | {self.board[1]} | {self.board[2]} | \n" \
               f"------------   *  ------------| \n" \
               f"| 3 | 4 | 5 |  *  | {self.board[3]} | {self.board[4]} | {self.board[5]} | \n" \
               f"------------   *  ------------| \n" \
               f"| 6 | 7 | 8 |  *  | {self.board[6]} | {self.board[7]} | {self.board[8]} | \n" \
               f"------------   *  ------------| \n"

    def _get_obs(self) -> List[str]:
        """
        Get the current observation

        Returns:
             The current board situation
        """
        return self.board

    def _add_action(self, action: int, actor_symbol: str) -> None:
        """
        Adds the given action to the bord, i.e., places the symbol of the actor at the index given as action.

        Args:
            action: action of the player (index where to put the corresponding symbol)
            actor_symbol: symbol of the actor which is added at the given board position (action)

        """
        self.board[action] = actor_symbol

    def _has_winner(self) -> bool:
        """
        Checks whether the current board situation is a win.

        Returns:
            True, if player with last move has won
        """

        # TODO
        # Hardcode all 8 possible end situations
        winning_situations = [
            # Rows
            self.board[:3],
            self.board[3:6],
            self.board[6:9],
            # Cols
            self.board[::3],
            self.board[1::3],
            self.board[2::3],
            # Diags
            self.board[::4],
            self.board[2::2][:-1],
        ]

        # Check if only one symbol occurs and if that symbol is mine: ["X", "X", " "] => {"X", " "} =! 1
        fulfilled = lambda field: len(set(field)) == 1 and field[0] == self.current_actor_symbol
        # Check for every situation
        return any(map(fulfilled, winning_situations))

    def _is_draw(self) -> bool:
        """
        Check whether the current board situation corresponds to a draw.

        Returns:
            True, if the board signals a draw
        """
        return all(field != ' ' for field in self.board)

    def _available_fields(self) -> List[int]:
        """
        Get the indexes of the available fields

        Returns:
            List of indexes of available fields
        """

        return [pos for pos, val in enumerate(self.board) if val == ' ']

    def reset(self) -> List[str]:
        """
        Resets the Environment to be ready for a new game

        Returns:
            the current observation (state of the board)
        """
        self.board = [' ' for _ in range(BOARD_SIZE)]
        self.done = False

        return self._get_obs()

    def step(self, action: int) -> Tuple[List[str], Rewards, bool]:
        """
        Do one step within the environment by applying the action.
        To do so, the following steps are required:
            1) Check the validity of the given action
            2) Add the given action to the board using the current actor symbol
            3) Check whether the last action produces a win or a draw and return observation, reward and done accordingly

        Args:
            action: Location on the board where the current player places his mark

        Returns:
            Observation - current board representation
            Reward - reward assigned to this step
            Done - boolean information whether the game is finished
        """

        # TODO
        # 1) Validity check
        if action not in self._available_fields():
            # return (self._get_obs(), Rewards.NO_REWARD.value, False)
            raise ValueError(f"Action {action} is not valid")


        # 2) Do action
        self._add_action(action, self.current_actor_symbol)

        # 3) Modify reward and done, return (Obs, Reward, Done)
        if self._has_winner():
            reward = Rewards.WINNER
            self.done = True
        elif self._is_draw():
            reward = Rewards.DRAW
            self.done = True
        else:
            reward = Rewards.NO_REWARD
            self.done = False

        # reward = Rewards.WINNER if self._has_winner() else Rewards.DRAW if self._is_draw() else Rewards.NO_REWARD
        # self.done = any([self._has_winner(), self._is_draw()])
        return (self._get_obs(), reward, self.done)

    def render(self, mode="human"):
        if mode == "human":
            return str(self)

    def set_current_actor_symbol(self, actor_symbol: str) -> None:
        """
        Set the given symbol as current actor symbol
        Args:
            actor_symbol: symbol of the current actor
        """
        assert actor_symbol in ['O', 'X']

        self.current_actor_symbol = actor_symbol


Des weiteren schreiben wir uns eine kleine Hilfsfunktion, die basierend auf einem beobachteten Spielzustand die noch verfügbaren Felder zurück gibt:

In [5]:
def available_fields_in_state(state: List[str]) -> List[int]:
    """
    Get the indexes of the available fields for the given state
    Returns:
         List of indexes of available fields
    """
    return [pos for pos, val in enumerate(state) if val == ' ']

<br>

Nachdem wir nun die Umgebung implementier haben, sollen als nächstes verschiedene Formen eines Spielers implementiert werden, u.a. MENACE.
Zunächst definieren wir die allgemeine Struktur eines Spielers:

In [6]:
from abc import ABC, abstractmethod
from typing import List

class NoughtsAndCrossesActor(ABC):
    """
    Abstract class of an actor for noughts-and-crosses
    """

    def __init__(self, actor_symbol: str = '-'):
        self.actor_symbol = actor_symbol

    @abstractmethod
    def start_game(self) -> None:
        """
        Preparation of the actor before the game starts.
        """

        raise NotImplementedError

    def get_action(self, state: List[str]) -> int:
        """
        Returns the action of the actor for the given board constellation.

        Args:
            state: current, observed state of the game board

        Returns:
            [int] index of the board field where the actor wants to place his mark
        """

        raise NotImplementedError

    @abstractmethod
    def win(self) -> None:
        """
        Is called when the actor has won and rewards the actor accordingly.
        """

        raise NotImplementedError

    @abstractmethod
    def draw(self) -> None:
        """
        Is called when the actor has played draw and rewards the actor accordingly.
        """

        raise NotImplementedError

    @abstractmethod
    def lose(self) -> None:
        """
        Is called when the actor has lost the game and punishes the actor accordingly.
        """

        raise NotImplementedError

    def print_stats(self):
        print(f"No stats available")

Damit später auch der trainierte MENACE-Agent zu einem Duell herausgefordert werden kann, implementieren wir einen menschlichen Spieler wie folgt:

In [7]:
class HumanPlayer(NoughtsAndCrossesActor):
    """
    Player that allows for human interactions.
    """

    def start_game(self):
        """
        Informs the player about starting match.
        """
        print("Get ready!")

    def get_action(self, state: List[str]) -> int:
        """
        Collects the player's action as console input.
        Player needs to input the position of the field where she/he wants to place his symbol.
        
        Args:
             state: the current, observed state of the board

        Returns:
            action of the human player
        """
        while True:
            move = input('Make a move: ')
            if move != "" and int(move) in available_fields_in_state(state=state):
                return int(move)
            print("Not a valid move")

    def win(self):
        """
        Informs the human player about his win.
        """
        print("You won!")

    def draw(self):
        """
        Informs the human player about the draw.
        """
        print("It's a draw.")

    def lose(self):
        """
        Informs the human player about his defeat.
        """
        print("You lose.")

Da wir MENACE aber nicht per Hand trainieren möchten, erstellen wir uns auch einen Agenten, der den nächsten Zug schlicht zufällig aus den noch verfügbaren Feldern wählt:

In [8]:
import random

class RandomPlayer(NoughtsAndCrossesActor):
    """
    A counterpart that simply draws a random action from the available fields
    of the present board constellation.
    """

    def start_game(self):
        pass

    def win(self):
        pass

    def draw(self):
        pass

    def lose(self):
        pass

    def get_action(self, state: List[str]) -> int:
        return random.choice(available_fields_in_state(state=state))

Zu guter Letzt fehlt uns nun nur noch die Implementierung eines `BasicMENACE`-Agenten basierend auf `NoughtsAndCrossesActor`.

### Aufgabe 2: Implementierung eines MENACE-Agenten

In [9]:
import math

class BasicMENACE(NoughtsAndCrossesActor):
    """
    Implementation of a basic MENACE actor that learns from his previous actions.
    """

    def __init__(self):
        super().__init__()
        self.matchboxes = dict()  # dictionary states: actions -> policies
        self.wins = 0
        self.draws = 0
        self.defeats = 0
        self.winning_shares = []
        self.actions_taken: List[Tuple[str, int]] = []

    def update_winning_shares(self) -> None:
        """
        Calculate the winning share after each 500 games.
        """
        num_matches = sum([self.wins, self.draws, self.defeats])
        if num_matches % 500 == 0:
            self.winning_shares.append(self.wins / num_matches)

    def start_game(self) -> None:
        """
        Prepare MENACE for a new match and reset his taken actions.
        They are gathered to update the mumbles in accordance to the result of the game.
        """
        self.actions_taken = []

    @staticmethod
    def _initialize_matchbox(state: List[str]) -> List[int]:
        """
        Initialize a new matchbox.
        On creation, a matchbox contains one mumble for each available field of the current board constellation.
        There as many types of mumbles as noughts-and-cross has fields.
        In dependence of the game progress, the amount of mumbles is multiplied to account
        for the number of outstanding actions.  The earlier the board state the more action options are required
        first round: 4 mumbles per available field , second round 3 mumbles...
        The input of the method is the representation of the current board.
        With the help of the function `available_fields_in_state`, the method should return a list of integers
        where each integer refers to the type of one mumble.

        Example:

            Zug 1 Runde 1 => 4  -> (9 Felder frei) / 2 = 4,5 -> 4
            Zug 2 Runde 1 => 4

            Zug 3 Runde 2 => 3 -> (7 Felder frei) / 2 = 3,5 -> 3
            Zug 4 Runde 2 

            Zug 5 Runde 3 => 2 ...
            Zug 6  ...
            Zug 7 => 1
            Zug 8
            Zug 9 => 0? 

            input:
                state = ["-", "X", "O", "-", "-", "-", "O", "X", "X"]

            output:
                [0, 3, 4, 5, 0, 3, 4, 5]
        """

        # TODO
        marbles = available_fields_in_state(state) # [0, 3, 4, 5]
        scale = math.floor(len(marbles) / 2) # 2

        # ? Whats better? Scale interval [1;5] or [0;4] with check for 0?
        if scale == 0:
            return marbles
        return marbles * scale

    def get_action(self, state: List[str]) -> int:
        """
        Returns the action MENACE decides to take given the current board constellation.
        If MENACE is not able to take an action, it returns -1.
        Therefore, two actions are required:
            1) check whether the given state is already known by MENACE, if not initialize a matchbox for it
            2) Get a random mumble of the corresponding checkbox for the given state

        Returns:
            [int] action chosen from MENACE
        """

        # TODO
        # 1) Check if state exists in matchboxes
        if str(state) not in self.matchboxes.keys():
            self.matchboxes[str(state)] = self._initialize_matchbox(state)
        
        actions = self.matchboxes[str(state)]
        # 2) Get random choice of possible actions
        return random.choice(actions) if len(actions) else -1

    def win(self) -> None:
        """
        Propagates the reward of winning the game to MENACE.
        Go through all actions taken for observed state within the current game
        Add the corresponding mumble (amount determined by Rewards.WINNER) for the taken action to the matchbox belonging to the observed state
        Increase winning counter and update winning shares
        """

        # TODO
        for state, marble in self.actions_taken:
            # Add mumbles to winning action matchboxes
            self.matchboxes[str(state)] = self.matchboxes[str(state)].extend([marble] * Rewards.WINNER)
        # Increase winning counter / update winning share
        self.wins += 1
        self.update_winning_shares()

    def draw(self) -> None:
        """
        Propagates the reward of a draw game to MENACE.
        Go through all actions taken for observed state within the current game
        Add the corresponding mumble (amount determined by Rewards.DRAW) for the taken action to the matchbox belonging to the observed state
        Increase draw counter and update winning shares
        """

        # TODO
        for state, marble in self.actions_taken:
            # Add mumbles to action matchboxes which led to draw
            self.matchboxes[str(state)] = self.matchboxes[str(state)].extend([marble] * Rewards.DRAW)
        # Increase draw counter / update winning share
        self.draws += 1 
        self.update_winning_shares()

    def lose(self) -> None:
        """
        Propagates the reward of losing the game to MENACE.
        """
        # go through all actions taken for an observed state within the current game
        for state, marble in self.actions_taken:
            # delete one of the corresponding marbles for the taken action
            # from the matchbox belonging to the observed state
            matchbox = self.matchboxes[state]
            del matchbox[matchbox.index(marble)]

        # update stats
        self.defeats += 1
        self.update_winning_shares()

    def print_stats(self) -> None:
        """
        Inform about how many board constellations MENACE knows and about the match results.
        :return:
        """

        print(f"Has seen {len(self.matchboxes.keys())} board constellations")
        print(f"Win/Draw/Defeat: {self.wins}/{self.draws}/{self.defeats}")

Nachdem wir nun bereits alle erforderlichen Komponenten implementiert haben, können wir uns nun um das Training von MENACE kümmern und uns danach einige Runden gegen ihn spielen.
Dafür verwenden wir die folgende Implementierung eines Tic-Tac-Toe Spiels:

In [10]:
class NoughtsAndCrosses:
    """
    Game of noughts and crosses.
    Player one always uses 'X' as symbol and player two always 'O'.
    """

    def __init__(self, player_one: NoughtsAndCrossesActor, player_two: NoughtsAndCrossesActor):
        self.player_one = player_one
        player_one.actor_symbol = 'X'
        self.player_two = player_two
        player_two.actor_symbol = 'O'
        self.env = NoughtsAndCrossesEnvironment()

    def play(self, silent: bool) -> None:
        """
        Play one match of noughts and crosses.

        :param silent: allows to deactivate the console output during the match (better for training)
        """

        # inform both players about the starting game
        self.player_one.start_game()
        self.player_two.start_game()

        state = self.env.reset()

        # repeat until no further action is possible
        while not self.env.done:

            # print current board constellation
            if not silent:
                print(self.env.render())

            # get action to take from player one
            action = self.player_one.get_action(state)

            # check whether the action is invalid
            if action == -1:
                self.player_one.lose()
                self.player_two.win()
                break

            self.env.set_current_actor_symbol(actor_symbol=self.player_one.actor_symbol)

            state, reward, done = self.env.step(action)

            # check whether the game is finished
            if reward == Rewards.WINNER:
                if not silent:
                    print(self.env.render())
                self.player_two.lose()
                self.player_one.win()
                break
            if reward == Rewards.DRAW:
                if not silent:
                    print(self.env.render())
                self.player_one.draw()
                self.player_two.draw()
                break

            # print current board constellation
            if not silent:
                print(self.env)

            # get action to take from player two
            action = self.player_two.get_action(state)

            # check whether the action is invalid
            if action == -1:
                self.player_two.win()
                self.player_two.lose()
                break

            self.env.set_current_actor_symbol(actor_symbol=self.player_two.actor_symbol)

            state, reward, done = self.env.step(action)

            # check whether the game is finished
            if reward == Rewards.WINNER:
                self.player_one.lose()
                self.player_two.win()
                if not silent:
                    print(self.env)
                break
            if reward == Rewards.DRAW:
                self.player_one.draw()
                self.player_two.draw()
                if not silent:
                    print(self.env)
                break

    def get_players(self) -> Tuple[NoughtsAndCrossesActor, NoughtsAndCrossesActor]:
        """
        Get the player instances of the game
        """
        return self.player_one, self.player_two

    def reset(self) -> None:
        """
        Refreshes the environment for a new game
        """
        self.env.reset()

Nun wollen wir damit MENACE einige Runden mittels des Zufallsspielers trainieren (spielt gerne etwas mit der Anzahl der Trainingsläufe herum).

In [11]:
# train MENACE with a randomly playing opponent
training_game = NoughtsAndCrosses(player_one=BasicMENACE(), player_two=RandomPlayer())
for i in range(75000):
    training_game.play(silent=True)
    training_game.reset()

In [12]:
# get the trained MENACE agent
menace, _ = training_game.get_players()

In [13]:
# print information about how MENACE has performed
menace.print_stats()

Has seen 2423 board constellations
Win/Draw/Defeat: 43880/9723/21397


In [14]:
# play some games against MENACE and see how well he behaves
manual_games = 5
for _ in range(manual_games):
    game = NoughtsAndCrosses(player_one=menace, player_two=HumanPlayer())
    game.play(silent=False)

Get ready!
| 0 | 1 | 2 |  *  |   |   |   | 
------------   *  ------------| 
| 3 | 4 | 5 |  *  |   |   |   | 
------------   *  ------------| 
| 6 | 7 | 8 |  *  |   |   |   | 
------------   *  ------------| 

| 0 | 1 | 2 |  *  |   |   |   | 
------------   *  ------------| 
| 3 | 4 | 5 |  *  |   |   | X | 
------------   *  ------------| 
| 6 | 7 | 8 |  *  |   |   |   | 
------------   *  ------------| 

| 0 | 1 | 2 |  *  |   |   |   | 
------------   *  ------------| 
| 3 | 4 | 5 |  *  |   | O | X | 
------------   *  ------------| 
| 6 | 7 | 8 |  *  |   |   |   | 
------------   *  ------------| 

| 0 | 1 | 2 |  *  |   |   |   | 
------------   *  ------------| 
| 3 | 4 | 5 |  *  |   | O | X | 
------------   *  ------------| 
| 6 | 7 | 8 |  *  | X |   |   | 
------------   *  ------------| 

| 0 | 1 | 2 |  *  |   |   |   | 
------------   *  ------------| 
| 3 | 4 | 5 |  *  |   | O | X | 
------------   *  ------------| 
| 6 | 7 | 8 |  *  | X |   | O | 
------------   *  ----------

KeyboardInterrupt: Interrupted by user

<br>

### Frage 1: Lass MENACE nun ein einige Male als zweiten Spieler antreten. Was fällt dir auf? Wie lässt sich das erklären?

> MENACE versucht immer zu gewinnen, Unentschieden macht für ihn keinen Unterschied und somit kann man ihm "Wege abschneiden".
Liegt möglicherweise an dem Reward für Draws.
> MENACE hat vermutlich viele Positionen, die man als Spieler 2 sieht, noch nicht gesehen und kann daher wahrscheinlich nicht "reaktiv" spielen. Liegt wahrscheinlich daran, dass er nicht als 2. Spieler gestartet ist.

### Frage 2: MENACE wurde bisher nur mittels eines zufällig agierenden Spielers trainiert, sicher gibt es hier noch bessere Möglichkeiten. Was fällt dir hierzu ein? Setze eine Möglichkeit um.

> - MENACE kann auch gegen sich selbst antreten.
> - Kann gegen Menschen antreten (Crowd Sourced)
> - Training gegen "Optimalen Spieler"
> - Mehr Training
> - Speicher für ideale Situationen einführen / Winning/Losing Positions

In [15]:
training_game = NoughtsAndCrosses(player_one=RandomPlayer(), player_two=BasicMENACE())
for i in range(75000):
    training_game.play(silent=True)
    training_game.reset()

_, menace = training_game.get_players()

print("Menace as player 2")
menace.print_stats()

training_game = NoughtsAndCrosses(player_one=RandomPlayer(), player_two=BasicMENACE())
for i in range(75000):
    training_game.play(silent=True)
    training_game.reset()

_, menace2 = training_game.get_players()


menace.wins = 0
menace.draws = 0
menace.defeats = 0
menace.winning_shares = []
menace2.wins = 0
menace2.draws = 0
menace2.defeats = 0
menace2.winning_shares = []


training_game = NoughtsAndCrosses(player_one=menace, player_two=RandomPlayer())
for i in range(75000):
    training_game.play(silent=True)
    training_game.reset()

menace, _ = training_game.get_players()

print("\nPretrained Menace as player 1")
menace.print_stats()


menace.wins = 0
menace.draws = 0
menace.defeats = 0
menace.winning_shares = []


training_game = NoughtsAndCrosses(player_one=menace, player_two=menace2)
for i in range(75000):
    training_game.play(silent=True)
    training_game.reset()

menace, menace2 = training_game.get_players()

print("\nPretrained Menace as player 1 vs Menace as Player 2")
menace.print_stats()
print()
menace2.print_stats()


menace.wins = 0
menace.draws = 0
menace.defeats = 0
menace.winning_shares = []
menace2.wins = 0
menace2.draws = 0
menace2.defeats = 0
menace2.winning_shares = []


training_game = NoughtsAndCrosses(player_one=menace2, player_two=menace)
for i in range(75000):
    training_game.play(silent=True)
    training_game.reset()

menace, menace2 = training_game.get_players()

print("\nMenace as player 1 vs Pretrained Menace as Player 2")
menace.print_stats()
print()
menace2.print_stats()


Menace as player 2
Has seen 2097 board constellations
Win/Draw/Defeat: 21737/9434/43829

Pretrained Menace as player 1
Has seen 4520 board constellations
Win/Draw/Defeat: 43685/9502/21813

Pretrained Menace as player 1 vs Menace as Player 2
Has seen 4520 board constellations
Win/Draw/Defeat: 43877/9479/21644

Has seen 2097 board constellations
Win/Draw/Defeat: 21644/9479/43877

Menace as player 1 vs Pretrained Menace as Player 2
Has seen 4520 board constellations
Win/Draw/Defeat: 43676/9699/21625

Has seen 4520 board constellations
Win/Draw/Defeat: 21625/9699/43676


### Frage 3: Welche Möglichkeiten fallen dir zur Verbesserung von MENACE ein? Was können wir anders/besser modellieren, damit das Lernen schneller bzw. besser klappt?

> - Identische Modellkonstellationen identifizieren (Rotationen/Spiegelung/Invertierung von Spielern (Kreuze/Kreise tauschen))
> - Rewards anpassen
> - Zufällig als ersten und zweiten Spieler antreten lassen (Stratified)
> - Winning/Losing Positions hardcoden
> - Matchbox System ersetzen?
> - Menace mit unterschiedlichen Policies gegeneinander spielen lassen (bspw. Hardcoded winning/losing positions vs. basic version)